In [1]:
import numpy as np
import pandas as pd
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor
import regex as re
Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [96]:
AMP_data_1of2 = pd.read_csv(Root + "/" + "Programing/Data/AMP_focussed_data/peptides.csv")
AMP_data_2of2 = pd.read_csv(Root + "/" + "Programing/Data/AMP_focussed_data/peptides(1).csv")

In [129]:
#merger the two parts of the dataset
DBAASP_data = pd.concat([AMP_data_1of2, AMP_data_2of2], axis = 0)
DBAASP_data = DBAASP_data.reset_index()

In [139]:
#extract the usefull columns
DBAASP_usefull = DBAASP_data[["ID", "SEQUENCE", "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE",
                              "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION",
                             "HEMOLITIC CYTOTOXIC ACTIVITY - UNIT", 'HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL' 
                              ]]


In [138]:
DBAASP_data[].value_counts()

Human erythrocytes                     6599
Rabbit erythrocytes                     382
Sheep erythrocytes                      369
Rat erythrocytes                        342
Mouse erythrocytes                      313
                                       ... 
Mouse retinal cells 661W                  1
Porcine macrophage cells 3D4/2            1
Rat intestinal epithelial cells IEC       1
Human corneal fibroblasts HTK             1
LB-EBV                                    1
Name: HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL, Length: 160, dtype: int64

Positive when:
>=5% cell death with <=10uM peptide
>=10% cell death with <=20uM peptide
>=15% cell death with <=50uM peptide
>=10% cell death with <=100uM peptide
>=30% cell death with <=200uM peptide
>=50% cell death with <=300uM peptide
neg if not or 
0% cell death with >=500uM
or <=...% or <...% cell death with >=500uM

In [140]:
#make the data workable
#by removing unknown values and empty values
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    'HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL'] == 'Human erythrocytes']
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"] != 'MEC']
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"] != 'EC50']
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"] != '-']
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"] != 'nan']
DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"
              ] = DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"].replace([
    "EC50", "IC50", 'LD50', 'CC50', 'LC50', 'MHC']
    , ["50%",'50%', '50%', '50%', '50%', '15%'])
DBAASP_usefull = DBAASP_usefull.dropna(subset=["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"])
DBAASP_usefull = DBAASP_usefull.dropna(subset=["HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"])
DBAASP_usefull = DBAASP_usefull.reset_index(drop=True)

In [146]:
DBAASP_usefull[0:60]

,ID,SEQUENCE,HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE,HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION,HEMOLITIC CYTOTOXIC ACTIVITY - UNIT,HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL,MW,CONCENTRATION_µM,conc_num,Lysis_value,toxic
0,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,50% Hemolysis,6,µM,Human erythrocytes,3140.87,6.00,6,50,1
1,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,15.6% Hemolysis,4,µM,Human erythrocytes,3140.87,4.00,4,15.6,1
2,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,23.1% Hemolysis,8,µM,Human erythrocytes,3140.87,8.00,8,23.1,1
3,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,33.1% Hemolysis,16,µM,Human erythrocytes,3140.87,16.00,16,33.1,1
4,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,43.6% Hemolysis,32,µM,Human erythrocytes,3140.87,32.00,32,43.6,1
5,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,74.3% Hemolysis,64,µM,Human erythrocytes,3140.87,64.00,64,74.3,1
6,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,87.4% Hemolysis,128,µM,Human erythrocytes,3140.87,128.00,128,87.4,1
7,14,GIWDTIKSMGKVFAGKILQNL,50% Hemolysis,90,µM,Human erythrocytes,2318.79,90.00,90,50,1
8,15,DSHAKRHHGYKRKFHEKHHSHRGY,50% Hemolysis,49±9.0,µM,Human erythrocytes,3035.32,49.00,49,50,1
9,15,DSHAKRHHGYKRKFHEKHHSHRGY,50% Hemolysis,>66,µM,Human erythrocytes,3035.32,66.00,66,50,1


In [141]:
#add molweight to covert ug/ml to uM
glob = GlobalDescriptor(DBAASP_usefull.SEQUENCE.tolist())
glob.calculate_MW(amide=True)
DBAASP_usefull['MW'] = glob.descriptor
#see what concentrations are in µg/ml instead of µM
mask = DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - UNIT"] == 'µg/ml'
#extract numerical to remove >, <, <= etc.
DBAASP_usefull["CONCENTRATION_µM"] = 0
for o in range(len(DBAASP_usefull)):
    if "-" in str(DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"][o]):
         DBAASP_usefull.loc[o, "conc_num"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
        "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"][o]))[1]
    else:
        DBAASP_usefull.loc[o, "conc_num"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
            "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"][o]))[0]
#calculate the µM
DBAASP_usefull["CONCENTRATION_µM"] = 0
DBAASP_usefull.loc[mask, "CONCENTRATION_µM"] = ((DBAASP_usefull[
    "conc_num"][mask]).astype(float)*1000) / DBAASP_usefull[mask]['MW']
#fill in values already in µM
mask2 = ~mask
DBAASP_usefull.loc[mask2, "CONCENTRATION_µM"] = DBAASP_usefull["conc_num"][mask2].astype(float)

In [142]:
#Extracting int for Lysis_value
for i in range(len(DBAASP_usefull)):
    if "-" in str(DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"][o]):
        DBAASP_usefull.loc[i, "Lysis_value"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
            "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"][i]))[1]
    else:
        DBAASP_usefull.loc[i, "Lysis_value"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
            "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"][i]))[0]

In [143]:
#select wether toxicity is positive or negative
#>=5% cell death with <=10uM peptide
#>=10% cell death with <=20uM peptide
#>=15% cell death with <=50uM peptide
#>=10% cell death with <=100uM peptide
#>=30% cell death with <=200uM peptide
#>=50% cell death with <=300uM peptide
#neg if not or 
#<=5% cell death with >=50uM
DBAASP_usefull["toxic"] = "~"
for v in range(len(DBAASP_usefull)):
    if float(DBAASP_usefull["Lysis_value"][v]) <= 5:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 50:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if float(DBAASP_usefull["Lysis_value"][v]) >= 5:
        if float(DBAASP_usefull["CONCENTRATION_µM"][v]) <= 10:
            DBAASP_usefull.loc[v, "toxic"] = 1
        else:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if float(DBAASP_usefull["Lysis_value"][v]) >= 10:
        if float(DBAASP_usefull["CONCENTRATION_µM"][v]) <= 20:
            DBAASP_usefull.loc[v, "toxic"] = 1
        else:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if float(DBAASP_usefull["Lysis_value"][v]) >= 15:
        if float(DBAASP_usefull["CONCENTRATION_µM"][v]) <= 50:
            DBAASP_usefull.loc[v, "toxic"] = 1
        else:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if float(DBAASP_usefull["Lysis_value"][v]) >= 20:
        if float(DBAASP_usefull["CONCENTRATION_µM"][v]) <= 100:
            DBAASP_usefull.loc[v, "toxic"] = 1
        else:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if float(DBAASP_usefull["Lysis_value"][v]) >= 30:
        if float(DBAASP_usefull["CONCENTRATION_µM"][v]) <= 200:
            DBAASP_usefull.loc[v, "toxic"] = 1
        else:
            DBAASP_usefull.loc[v, "toxic"] = 0            
    if float(DBAASP_usefull["Lysis_value"][v]) >= 50:
        if float(DBAASP_usefull["CONCENTRATION_µM"][v]) <= 300:
            DBAASP_usefull.loc[v, "toxic"] = 1
        else:
            DBAASP_usefull.loc[v, "toxic"] = 0 

In [154]:
#happenn formating
DBAASP_usefull["toxic"] = "~"
for v in range(len(DBAASP_usefull)):
    if float(DBAASP_usefull["Lysis_value"][v]) <= 5:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 30:
            DBAASP_usefull.loc[v, "toxic"] = 0        
    if 5 <= float(DBAASP_usefull["Lysis_value"][v]) <= 10:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 60:
            DBAASP_usefull.loc[v, "toxic"] = 0   
    if 10 <= float(DBAASP_usefull["Lysis_value"][v]) <= 15:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 90:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if 15 <= float(DBAASP_usefull["Lysis_value"][v]) <= 20:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 120:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if 20 <= float(DBAASP_usefull["Lysis_value"][v]) <= 25:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 180:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if 25 <= float(DBAASP_usefull["Lysis_value"][v]) <= 30:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 180:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if 30 <= float(DBAASP_usefull["Lysis_value"][v]) <= 35:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 210:
            DBAASP_usefull.loc[v, "toxic"] = 0         
    if 35 <= float(DBAASP_usefull["Lysis_value"][v]) <= 40:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 240:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if 40 <= float(DBAASP_usefull["Lysis_value"][v]) <= 45:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 270:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if float(DBAASP_usefull["Lysis_value"][v]) >= 50:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 300:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 55:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 330:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 60:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 360:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 65:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 390:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 70:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 420:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 75:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 450:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 80:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 480:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 85:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 510:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 90:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 540:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 95:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 570:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 100:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 600:
            DBAASP_usefull.loc[v, "toxic"] = 1 

In [155]:
#remove rows were toxicity is unclear
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull["toxic"] != '~']
DBAASP_usefull = DBAASP_usefull.reset_index(drop=True)
#merge same peptides
#where majority vote decides toxicity and a draw gives "~"
peptides = DBAASP_usefull["SEQUENCE"].unique()
DBAASP_final = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(DBAASP_usefull)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if DBAASP_usefull["SEQUENCE"][a] == peptides[n]:
            toxic.append(DBAASP_usefull["toxic"][a])
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                DBAASP_final.loc[n, "toxic"] = "~"
            if toxic_count < len(toxic)/2:
                DBAASP_final.loc[n, "toxic"] = 0
            if toxic_count > len(toxic)/2:
                DBAASP_final.loc[n, "toxic"] = 1

In [151]:
DBAASP_usefull

,ID,SEQUENCE,HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE,HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION,HEMOLITIC CYTOTOXIC ACTIVITY - UNIT,HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL,MW,CONCENTRATION_µM,conc_num,Lysis_value,toxic
0,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,50% Hemolysis,6,µM,Human erythrocytes,3140.87,6.00,6,50,0
1,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,74.3% Hemolysis,64,µM,Human erythrocytes,3140.87,64.00,64,74.3,1
2,11,RVKRVWPLVIRTVIAGYNLYRAIKKK,87.4% Hemolysis,128,µM,Human erythrocytes,3140.87,128.00,128,87.4,1
3,14,GIWDTIKSMGKVFAGKILQNL,50% Hemolysis,90,µM,Human erythrocytes,2318.79,90.00,90,50,0
4,15,DSHAKRHHGYKRKFHEKHHSHRGY,50% Hemolysis,49±9.0,µM,Human erythrocytes,3035.32,49.00,49,50,0
...,...,...,...,...,...,...,...,...,...,...,...
4016,18283,KRIWQRIKDFLR,50% Hemolysis,25,µM,Human erythrocytes,1658.01,25.00,25,50,0
4017,18284,KRIWQRIKDF,50% Hemolysis,>200,µM,Human erythrocytes,1388.67,200.00,200,50,0
4018,18285,KRIWQRIK,50% Hemolysis,>200,µM,Human erythrocytes,1126.41,200.00,200,50,0
4019,18286,KRIWQR,50% Hemolysis,>200,µM,Human erythrocytes,885.08,200.00,200,50,0


In [156]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from lazypredict.Supervised import LazyClassifier
from DNN_features import *
test2 = AAC_df(DBAASP_final.seq)
train_X, val_X, train_y, val_y = train_test_split(test2, DBAASP_final.toxic, random_state = 445)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
#models_train,predictions_train = clf.fit(train_X, train_X, train_y, train_y)
models,predictions = clf.fit(train_X, val_X, train_y, val_y)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  6.22it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,matthews_corrcoef,Time Taken
Model,,,,,,
ExtraTreesClassifier,0.83,0.83,0.83,0.83,0.65,0.38
RandomForestClassifier,0.82,0.82,0.82,0.82,0.64,0.46
LGBMClassifier,0.80,0.80,0.80,0.80,0.60,0.13
XGBClassifier,0.80,0.80,0.80,0.80,0.59,0.19
NuSVC,0.78,0.78,0.78,0.78,0.56,0.29
LabelPropagation,0.78,0.78,0.78,0.78,0.56,0.49
LabelSpreading,0.78,0.78,0.78,0.78,0.55,0.61
BaggingClassifier,0.78,0.77,0.77,0.78,0.55,0.15
SVC,0.77,0.77,0.77,0.77,0.53,0.21


In [158]:
DBAASP_final.to_csv(Root + "/Programing/Data/AMP_focussed_data/DBAASP_data.csv",index = False, header=True)

In [157]:
DBAASP_final.toxic.value_counts()

0.00    1652
1.00    1461
Name: toxic, dtype: int64